# The Seagull Story
In this project we aim to fine-tune a BERT-like transformer to solve a Natural Language Inference task. The model is asked `yes`/`no` questions and has to answer `yes`, `no`, or `irrelevant` based on a given story.

## The NLI task
*Natural Language Inference*, also known as *textual entailment*, is the task of determining whether a *hypothesis* is true (**entailment**), false (**contradiction**), or undetermined (**neutral**) given a *premise*.

In more precise terms, the *premise* $p$ entails the *hypothesis* $h$ ($p \implies h$) if and only if, typically, a human reading $p$ would be justified in inferring the proposition expressed by $h$ from the proposition expressed by $p$. This is a more relaxed definition than the pure *logical entailment*. The **relation is directional** because even if $p \implies h$, the reverse $h \implies p$ is much less certain.

Determining whether this relationship holds is an informal task, one which sometimes overlaps with the formal tasks of formal semantics. In this context, however, we want to tackle this task **using a BERT-like transformer**, which is now state-of-the-art for all kinds of NLP tasks.

Read more on [Textual entailment][1].

[1]:https://en.wikipedia.org/wiki/Textual_entailment

### The choice of BERT
When it comes to solving natural language tasks, the BERT and GPT models have come to the fore due to their innovative architectures. Both are based on the transformer architecture, which is based on the attention mechanism, discovered by Google in 2017 and published in the paper [Attention is All You Need](https://arxiv.org/abs/1706.03762) by Vaswani et al.

Unlike GPT, BERT is trained to predict missing words in text, using both left and right contexts. This implies that BERT is a bidirectional moddel, and thus is superior for tasks that require understanding the context and nuances of language. In other words, BERT is better suited to tackle our task.

Read more on [Differences Between GPT and BERT][2].

[2]:https://www.geeksforgeeks.org/differences-between-gpt-and-bert/

### The choice of a BERT-like model
